<a href="https://colab.research.google.com/github/sakamoto-hands-on/Python_Practical_DataAnalysis100/blob/master/05_Predicting_LeavingCustomers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#スポーツジムの退会しそうなユーザを予測

In [0]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

当月と過去1か月の利用回数データを作成

In [4]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
  tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
  tmp.rename(columns={"count":"count_0"}, inplace=True)
  tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
  del tmp_before["年月"]
  tmp_before.rename(columns={"count":"count_1"}, inplace=True)
  tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
  uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


退会したユーザを絞り込んで、End_Date列の1か月前の年月を取得し、uselogとcustomer_id、年月をキーにして結合

In [5]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"]=None
exit_customer["end_date"]=pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
  exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

33851


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


欠損値を削除

In [6]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,1.0,ナイト,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


##継続顧客のデータを作成

In [7]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


退会データが1104件に対して、継続顧客データは27422件と不均衡
継続顧客も、顧客あたり1件になるようにアンダーサンプリング
データをシャッフルして、重複を削除

In [8]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201812,GD106268,5,4.0,XXXXX,C01,F,2016-12-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.750000,5.0,6.0,2.0,1.0,2019-04-30,28.0
1,201903,HD384128,8,6.0,XXX,C03,F,2018-05-02,NaN,CA2,0.0,ナイト,6000.0,入会費半額,7.727273,8.0,9.0,6.0,1.0,2019-04-30,11.0
2,201807,TS948747,5,6.0,XX,C01,F,2017-06-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.333333,5.0,8.0,3.0,1.0,2019-04-30,22.0
3,201807,PL876696,7,4.0,XXXX,C02,F,2016-10-01,NaN,CA1,0.0,デイタイム,7500.0,通常,5.333333,5.0,8.0,1.0,1.0,2019-04-30,30.0
4,201805,HI764799,3,5.0,XXX,C01,M,2016-04-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.333333,4.5,6.0,2.0,1.0,2019-04-30,36.0


In [9]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201812,GD106268,5,4.0,XXXXX,C01,F,2016-12-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.750000,5.0,6.0,2.0,1.0,2019-04-30,28.0,NaN
1,201903,HD384128,8,6.0,XXX,C03,F,2018-05-02,NaN,CA2,0.0,ナイト,6000.0,入会費半額,7.727273,8.0,9.0,6.0,1.0,2019-04-30,11.0,NaN
2,201807,TS948747,5,6.0,XX,C01,F,2017-06-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.333333,5.0,8.0,3.0,1.0,2019-04-30,22.0,NaN
3,201807,PL876696,7,4.0,XXXX,C02,F,2016-10-01,NaN,CA1,0.0,デイタイム,7500.0,通常,5.333333,5.0,8.0,1.0,1.0,2019-04-30,30.0,NaN
4,201805,HI764799,3,5.0,XXX,C01,M,2016-04-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.333333,4.5,6.0,2.0,1.0,2019-04-30,36.0,NaN


##予測する月の在籍期間を作成する

In [10]:
predict_data["period"]=0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"]=pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
  delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
  predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201812,GD106268,5,4.0,XXXXX,C01,F,2016-12-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.750000,5.0,6.0,2.0,1.0,2019-04-30,28.0,NaN,24,2018-12-01
1,201903,HD384128,8,6.0,XXX,C03,F,2018-05-02,NaN,CA2,0.0,ナイト,6000.0,入会費半額,7.727273,8.0,9.0,6.0,1.0,2019-04-30,11.0,NaN,9,2019-03-01
2,201807,TS948747,5,6.0,XX,C01,F,2017-06-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.333333,5.0,8.0,3.0,1.0,2019-04-30,22.0,NaN,13,2018-07-01
3,201807,PL876696,7,4.0,XXXX,C02,F,2016-10-01,NaN,CA1,0.0,デイタイム,7500.0,通常,5.333333,5.0,8.0,1.0,1.0,2019-04-30,30.0,NaN,21,2018-07-01
4,201805,HI764799,3,5.0,XXX,C01,M,2016-04-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.333333,4.5,6.0,2.0,1.0,2019-04-30,36.0,NaN,25,2018-05-01


##欠損値を除去

In [11]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               242
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [12]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2652
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2652
period                  0
now_date                0
dtype: int64

##文字列型の変数を処理できるように整形する

In [13]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,F,4.0,1.0,24,0.0
1,入会費半額,ナイト,F,6.0,1.0,9,0.0
2,通常,オールタイム,F,6.0,1.0,13,0.0
3,通常,デイタイム,F,4.0,1.0,21,0.0
4,通常,オールタイム,M,5.0,1.0,25,0.0


カテゴリカル変数を用いてダミー変数を作成

In [14]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,4.0,1.0,24,0.0,0,0,1,1,0,0,1,0
1,6.0,1.0,9,0.0,1,0,0,0,0,1,1,0
2,6.0,1.0,13,0.0,0,0,1,1,0,0,1,0
3,4.0,1.0,21,0.0,0,0,1,0,1,0,1,0
4,5.0,1.0,25,0.0,0,0,1,1,0,0,0,1


get_dummiesによって、簡単にダミー変数が作られるが、多重共線性を避けるため、不要な変数は削除

In [15]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,4.0,1.0,24,0.0,0,0,1,0,1
1,6.0,1.0,9,0.0,1,0,0,0,1
2,6.0,1.0,13,0.0,0,0,1,0,1
3,4.0,1.0,21,0.0,0,0,0,1,1
4,5.0,1.0,25,0.0,0,0,1,0,0


##決定木を用いて退会予測モデルを作成

In [16]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0.
 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1.

In [17]:
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
617,1.0,1.0
1259,0.0,0.0
1050,1.0,1.0
707,1.0,0.0
423,1.0,1.0


##予測モデルの評価を行い、モデルのチューニング

In [18]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8878326996197718


In [19]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8878326996197718
0.9803548795944234


過学習を起こしてしまっているので、モデルをチューニング
決定木の木構造の深さを浅くする

In [20]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9163498098859315
0.926489226869455


##モデルに寄与している変数を確認

In [22]:
importance = pd.DataFrame({"feature_names": X.columns, "coefficient": model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.339574
1,routine_flg,0.133578
2,period,0.525247
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000426
6,class_name_デイタイム,0.000000
7,gender_F,0.001174


##顧客の退会を予測する

In [0]:
count_1 = 5
routine_flg =1
period = 20
campaign_name ="入会費半額"
class_name = "オールタイム"
gender = "F"

In [0]:
if campaign_name == "入会費半額":
  campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
  campaign_name_list = [0, 1]
elif campaign_name == "通常":
  campaign_name_list = [0, 0]
if class_name == "オールタイム":
  class_name_list = [1, 0]
elif class_name == "デイタイム":
  class_name_list = [0, 1]
elif class_name == "ナイト":
  class_name_list = [0, 0]
if gender == "F":
  gender_list = [1]
elif gender == "M":
  gender_list =[0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [38]:
input_data

[5, 1, 20, 1, 0, 1, 0, 1]

In [39]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[0.]
[[0.87878788 0.12121212]]
